# Create Bottleneck features

---

Before supplying an image to a pre-trained network in Keras, there are some required preprocessing steps. When using TensorFlow as backend, Keras Convolutional Neural Networks (CNNs) require a 4D-array as input, with shape `(nb_images, rows, columns, channels)`, where `nb_images` corresponds to the total number of images, and rows, columns, and channels correspond to the number of `rows`, `columns`, and `channels` for each image, respectively.

## 1. Load Files and Targets
The images from the training, validation and test datasets are first loaded.

In [ ]:
from keras.applications.vgg16 import preprocess_input
import numpy as np
from common import *

train_ids, train_targets = load_dataset('data/preprocess/train.npz')
valid_ids, valid_targets = load_dataset('data/preprocess/valid.npz')
test_ids, test_targets = load_dataset('data/preprocess/test.npz')

train_files = np.array(['data/train_photos/' + str(i) + '.jpg' for i in train_ids])
valid_files = np.array(['data/train_photos/' + str(i) + '.jpg' for i in valid_ids])
test_files = np.array(['data/train_photos/' + str(i) + '.jpg' for i in test_ids])

print('There are %d images in the training dataset' % len(train_files))
print('There are %d images in the validation dataset' % len(valid_files))
print('There are %d images in the test dataset' % len(test_files))

Each image is resized to a square image that is 224 x 224 pixels. Next, the image is converted to a 4D-array. Since, we are working with color images (`channels` = 3, RGB), the 4D-array has shape (1, 224, 224, 3).

The `paths_to_tensor` function used in the cell below takes a numpy array of string-valued image paths as input and returns a 4D-array with shape (`nb_images`, 224, 224, 3).

The idea is to compare the performances of various deep learning models on the same dataset and select the most promising one for this project. For this reason, only a small fraction of the data are considered to produce the bottleneck features. The models are compared in this [notebook](compare.ipynb).

In [ ]:
nb_train, nb_valid, nb_test = (30000,3000,3000)

train_tensors = preprocess_input(paths_to_tensor(train_files[:nb_train])).astype('float32')/255
valid_tensors = preprocess_input(paths_to_tensor(valid_files[:nb_valid])).astype('float32')/255
test_tensors = preprocess_input(paths_to_tensor(test_files[:nb_test])).astype('float32')/255

## 2. Bottleneck Features

### A. VGG16
#### - $\alpha$ Import the VGG16 Model, with the Final Fully-Connected Layers Removed

When performing transfer learning, we need to remove the final layers of the network, as they are too specific to the ImageNet database.  This is accomplished in the code cell below.

In [ ]:
from keras.applications.vgg16 import VGG16

model = VGG16(include_top=False)
model.summary()

#### - $\beta$ Extract Output of Final Max Pooling Layer
Now, the network stored in `model` is a truncated version of the VGG16 network, where the final three fully-connected layers have been removed.  In this case, `model.predict` returns a 3D array (with dimensions 7 x 7 x 512) corresponding to the final max pooling layer of VGG16.  The dimensionality of the obtained output from passing the 4D-array through the model is `(nb_images, 7, 7, 512)`.

In [ ]:
train_vgg16 = model.predict(train_tensors)
valid_vgg16 = model.predict(valid_tensors)
test_vgg16  = model.predict(test_tensors)

In [ ]:
np.savez('data/bottleneck_features/yelp_vgg16.npz', 
         train_features=train_vgg16, train_targets=train_targets[:nb_train],
         valid_features=valid_vgg16, valid_targets=valid_targets[:nb_valid],
         test_features=test_vgg16, test_targets=test_targets[:nb_test])

### B. Xception
#### - $\alpha$ Import the Xception Model, with the Final Fully-Connected Layers Removed
The same procedure is applied for the Xception deep learning model.

In [ ]:
from keras.applications.xception import Xception

model = Xception(include_top=False)
model.summary()

#### - $\beta$ Extract Output of Final Activation Layer

In [ ]:
train_xception = model.predict(train_tensors)
valid_xception = model.predict(valid_tensors)
test_xception  = model.predict(test_tensors)

In [ ]:
np.savez('data/bottleneck_features/yelp_xception.npz', 
         train_features=train_xception, train_targets=train_targets[:nb_train],
         valid_features=valid_xception, valid_targets=valid_targets[:nb_valid],
         test_features=test_xception, test_targets=test_targets[:nb_test])

### C. ResNet50
#### - $\alpha$ Import the ResNet50 Model, with the Final Fully-Connected Layers Removed
The same procedure is applied for the ResNet50 deep learning model.

In [ ]:
from keras.applications.resnet50 import ResNet50

model = ResNet50(include_top=False)
model.summary()

#### - $\beta$ Extract Output of Final Average Pooling Layer

In [ ]:
train_resnet50 = model.predict(train_tensors)
valid_resnet50 = model.predict(valid_tensors)
test_resnet50  = model.predict(test_tensors)

In [ ]:
np.savez('data/bottleneck_features/yelp_resnet50.npz', 
         train_features=train_resnet50, train_targets=train_targets[:nb_train],
         valid_features=valid_resnet50, valid_targets=valid_targets[:nb_valid],
         test_features=test_resnet50, test_targets=test_targets[:nb_test])

### D. InceptionV3
#### - $\alpha$ Import the Inception Model, with the Final Fully-Connected Layers Removed
The same procedure is applied for the Inception deep learning model.

In [ ]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(include_top=False)
model.summary()

#### - $\beta$ Extract Output of Final Concatenate Layer

In [ ]:
train_inception = model.predict(train_tensors)
valid_inception = model.predict(valid_tensors)
test_inception  = model.predict(test_tensors)

In [ ]:
np.savez('data/bottleneck_features/yelp_inception.npz', 
         train_features=train_inception, train_targets=train_targets[:nb_train],
         valid_features=valid_inception, valid_targets=valid_targets[:nb_valid],
         test_features=test_inception, test_targets=test_targets[:nb_test])